## Federated data integration testing

In [1]:
import sys
import os
sys.path.append("/data/Beatriz/Doctorado GR/ADL_platform/S-ADL")


##### Example of static data set for testing

In [2]:
from flexanomalies.utils.load_data import load_and_split_dot_mat
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/datasets/data/shuttle.mat", 0.3
)


2025-02-03 19:16:10.237362: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 19:16:10.238460: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 19:16:10.241563: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 19:16:10.249948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738606570.263671   12967 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738606570.26

#### Tests with different models of the flex-anomaly platform

In [3]:
from SADL.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "isolationForest",
    "contamination":0.1,
    "label_parser": None,
    "n_estimators": 100, 
    "n_rounds": 10,
    "n_clients":5,
    
}

modelIforest = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelIforest.get_params())

Federated Params:{'n_rounds': 10, 'n_clients': 5} 
 Model Params:{'algorithm_': 'isolationForest', 'contamination': 0.1, 'label_parser': None, 'n_estimators': 100}
{'algorithm_': 'isolationForest', 'contamination': 0.1, 'label_parser': None, 'n_estimators': 100}
odict_values([<Parameter "self">, <Parameter "n_estimators=100">, <Parameter "max_samples=1000">, <Parameter "contamination=0.1">, <Parameter "max_features=1.0">, <Parameter "bootstrap=False">, <Parameter "n_jobs=1">, <Parameter "behaviour='old'">, <Parameter "random_state=None">, <Parameter "model_path=''">, <Parameter "verbose=0">])
{'n_estimators': 100, 'contamination': 0.1}
Inspecting model's attributes:
contamination: 0.1
n_estimators: 100
max_samples: 1000
max_features: 1.0
bootstrap: False
n_jobs: 1
behaviour: old
random_state: None
verbose: 0
model_path: 
model: IsolationForest(contamination=0.1, max_samples=1000, n_jobs=1)
algorithm_: isolationForest
label_parser: None
{'label_parser': None, 'algorithm_': 'isolationFor

In [4]:
modelIforest.fit(X_train,y_train)


Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 3

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 4

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 5

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 6

Training model at client.
Training model at client.
Training model at client.
Training 

In [5]:
scores_pred = modelIforest.decision_function(X_train)
print(scores_pred)
modelIforest.predict(X_train)

[0.40890258 0.33228159 0.33283597 ... 0.40204865 0.34113684 0.33959725]
Inspecting model's attributes:
contamination: 0.1
n_estimators: 100
max_samples: 1000
max_features: 1.0
bootstrap: False
n_jobs: 1
behaviour: old
random_state: None
verbose: 0
model_path: 
model: IsolationForest(contamination=0.1, max_samples=1000, n_jobs=1)
algorithm_: isolationForest
label_parser: None


In [6]:
print(modelIforest.model.d_scores_ == scores_pred)
modelIforest.model.labels_


[ True  True  True ...  True  True  True]


array([1., 0., 0., ..., 1., 0., 0.])

In [7]:
modelIforest.evaluate(X_test,y_test)

Acc: 97.217% 

Precision: 0.736 

F1score: 0.841 

Recall: 0.981 

AUC_ROC: 0.976 



In [4]:
from SADL.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "autoencoder",
    "contamination":0.1,
    "label_parser": None,
     "epochs": 50,
     "input_dim": 9,
     "batch_size": 32,
     "neurons": [8, 4, 8],
     "hidden_act": ['relu', 'relu', 'relu'],
     "n_clients":5,
     "n_rounds":10,
     }

modelAE = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelAE.get_params())


Federated Params:{'n_clients': 5, 'n_rounds': 10, 'X_train': array([[38,  0, 76, ..., 38, 37,  0],
       [51,  0, 81, ..., 30, 32,  2],
       [37,  0, 77, ..., 40, 59, 18],
       ...,
       [37,  0, 76, ..., 39, 61, 22],
       [42,  4, 90, ..., 48, 49,  0],
       [37,  0, 77, ..., 40, 51, 12]]), 'y_train': array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])} 
 Model Params:{'algorithm_': 'autoencoder', 'contamination': 0.1, 'label_parser': None, 'epochs': 50, 'input_dim': 9, 'batch_size': 32, 'neurons': [8, 4, 8], 'hidden_act': ['relu', 'relu', 'relu']}
{'algorithm_': 'autoencoder', 'contamination': 0.1, 'label_parser': None, 'epochs': 50, 'input_dim': 9, 'batch_size': 32, 'neurons': [8, 4, 8], 'hidden_act': ['relu', 'relu', 'relu']}
odict_values([<Parameter "self">, <Parameter "input_dim">, <Parameter "neurons">, <Parameter "model_path=''">, <Parameter "callbacks=[]">, <Parameter "hidden_act='relu'">, <Parameter "output_act='linear'">, <Parameter

In [ ]:
modelAE.fit(X_train,y_train)


In [11]:
scores_pred = modelAE.decision_function(X_train)
print("Scores",scores_pred)
modelAE.predict(X_train)

1074/1074 ━━━━━━━━━━━━━━━━━━━━ 1s 476us/step
Scores [0.07801598 0.15745349 0.08044425 ... 0.05917886 0.11189506 0.05946571]
Inspecting model's attributes:
contamination: 0.1
input_dim: 9
neurons: [8, 4, 8]
model_path: 
w_size: None
n_pred: 1
callbacks: [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7368405b7640>]
hidden_act: ['relu', 'relu', 'relu']
output_act: linear
loss: mse
validation_size: 0.2
batch_size: 32
epochs: 50
optimizer: adam
preprocess: True
scaler: StandardScaler()
model: <Sequential name=sequential_2, built=True>
algorithm_: autoencoder
label_parser: None
d_scores_: [0.07801598 0.15745349 0.08044425 ... 0.05917886 0.11189506 0.05946571]
labels_: [0. 0. 0. ... 0. 0. 0.]
1074/1074 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step


array([[-0.654824  ,  0.00584355, -0.99394816, ..., -0.02929544,
        -0.51234984, -0.42424423],
       [ 0.3091159 ,  0.06364918, -0.45604682, ..., -0.62068903,
        -0.7887534 , -0.3291298 ],
       [-0.73183453,  0.00199485, -0.8784186 , ...,  0.13044083,
         0.53195655,  0.38427576],
       ...,
       [-0.7301675 ,  0.00493526, -1.0173206 , ...,  0.03971636,
         0.6169077 ,  0.5091003 ],
       [-0.35435343,  0.03557914,  0.57791436, ...,  0.7121097 ,
         0.00618351, -0.39747688],
       [-0.73751783, -0.01134115, -0.92882925, ...,  0.09666133,
         0.14811838,  0.07195055]], dtype=float32)

In [12]:
modelAE.model.d_scores_ ==scores_pred
modelAE.model.labels_


array([0., 0., 0., ..., 0., 0., 0.])

In [39]:
modelAE.evaluate(X_test,y_test)

461/461 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step
Acc: 94.297% 

Precision: 0.567 

F1score: 0.665 

Recall: 0.805 

AUC_ROC: 0.879 



In [4]:
from SADL.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "clusterAnomaly",
    "contamination":0.1,
    "label_parser": None,
    "n_clusters": 4,
    "n_clients":5,
    "n_rounds":10,
       
}

modelCluster = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelCluster.get_params())

Federated Params:{'n_clients': 5, 'n_rounds': 10, 'X_train': array([[43,  0, 76, ..., 33, 35,  2],
       [56,  0, 79, ..., 23, 22,  0],
       [41,  1, 86, ..., 45, 47,  2],
       ...,
       [45,  3, 86, ..., 40, 42,  2],
       [38,  4, 83, ..., 45, 44,  0],
       [45,  0, 74, ..., 29, 30,  2]]), 'y_train': array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])} 
 Model Params:{'algorithm_': 'clusterAnomaly', 'contamination': 0.1, 'label_parser': None, 'n_clusters': 4}
{'algorithm_': 'clusterAnomaly', 'contamination': 0.1, 'label_parser': None, 'n_clusters': 4}
odict_values([<Parameter "self">, <Parameter "n_clusters">, <Parameter "seed=None">, <Parameter "init_centroids='random'">, <Parameter "max_iter=100">, <Parameter "tol=0.0001">, <Parameter "n_init=1">, <Parameter "verbose=True">, <Parameter "algorithm='lloyd'">, <Parameter "model_path=''">, <Parameter "contamination=0.1">])
{'n_clusters': 4, 'contamination': 0.1}
Inspecting model's attributes:


In [6]:
modelCluster.fit(X_train,y_train)



Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 3

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 4

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 5

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 6

Training model at client.
Training model at client.
Training model at client.
Training 

In [ ]:
scores_pred = modelCluster.decision_function(X_train)
print(scores_pred)
modelCluster.predict(X_train)

In [10]:
modelCluster.evaluate(X_test,y_test)

Acc: 93.727% 

Precision: 0.529 

F1score: 0.628 

Recall: 0.772 

AUC_ROC: 0.861 



In [10]:
from SADL.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "pcaAnomaly",
    "contamination":0.1,
    "label_parser": None,
    "preprocess":False,  
    "n_components": 4,
    "n_clients":5,
    "n_rounds":10,      
}

modelPca = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelPca.get_params())

Federated Params:{'n_clients': 5, 'n_rounds': 10, 'X_train': array([[ 37,   0,  76, ...,  39,  52,  14],
       [ 38,   4,  77, ...,  39,  39,   0],
       [ 77,   0,  81, ...,   4, 125, 120],
       ...,
       [ 46,   4,  93, ...,  47,  47,   0],
       [ 57,   4,  88, ...,  31,  31,   0],
       [ 49,   1,  77, ...,  28,  28,   0]]), 'y_train': array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])} 
 Model Params:{'algorithm_': 'pcaAnomaly', 'contamination': 0.1, 'label_parser': None, 'preprocess': False, 'n_components': 4}
{'algorithm_': 'pcaAnomaly', 'contamination': 0.1, 'label_parser': None, 'preprocess': False, 'n_components': 4}
odict_values([<Parameter "self">, <Parameter "n_components=None">, <Parameter "n_selected_components=None">, <Parameter "copy=True">, <Parameter "whiten=False">, <Parameter "svd_solver='auto'">, <Parameter "tol=0.0">, <Parameter "iterated_power='auto'">, <Parameter "random_state=None">, <Parameter "model_path=''">, <Param

In [11]:
modelPca.fit(X_train,y_train)


Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 3

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 4

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 5

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 6

Training model at client.
Training model at client.
Training model at client.
Training 

In [13]:
scores_pred = modelPca.decision_function(X_train)
print(scores_pred)
modelPca.predict(X_train)

[ 5692.63513956  5568.7802977  10746.28820812 ...  6724.74794335
  6480.80792311  5693.50007755]
Inspecting model's attributes:
contamination: 0.1
n_components: 4
n_selected_components: None
copy: True
whiten: False
svd_solver: auto
tol: 0.0
iterated_power: auto
random_state: None
weighted: (True,)
preprocess: False
model_path: 
scaler: StandardScaler()
model: PCA(n_components=4)
algorithm_: pcaAnomaly
label_parser: None
n_components_: 4
components_: [[ 2.36809510e-04 -7.06239103e-03 -6.62156800e-04  8.30378679e-04
   9.41751416e-03  9.99693583e-01 -8.75837290e-04 -1.00971026e-02
  -9.07926023e-03]
 [ 1.34815145e-02  9.99315375e-01 -2.57277491e-04  3.03622446e-04
  -4.86400742e-03  7.26558135e-03 -1.36914587e-02  4.69537632e-03
   1.83713153e-02]
 [ 4.64269249e-02 -1.36959355e-02  8.11142172e-03  6.01209371e-01
  -2.01363284e-01  7.92225233e-03 -3.79276719e-02  2.11995111e-01
   2.48832126e-01]
 [-3.77541752e-02 -3.07740252e-03  2.57961294e-02  2.14951513e-01
  -3.63968713e-01  7.61007

In [15]:
modelPca.evaluate(X_test,y_test)

Acc: 96.660% 

Precision: 0.686 

F1score: 0.804 

Recall: 0.972 

AUC_ROC: 0.969 

